In [11]:
from autoencoders import *
from pdb_utils import *

import pickle
import numpy as np
import random

import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from functools import partial
import sys

from Bio import PDB
import numpy as np
import os
import cv2

import matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV

import time

In [12]:
with open('pickle files/fold_groups.pkl', 'rb') as f:
    fold_dict = pickle.load(f)
with open('pickle files/label_dict.pkl', 'rb') as f:
    label_dict = pickle.load(f)
with open('pickle files/test_labels.pkl', 'rb') as f:
    test_label_dict = pickle.load(f)

In [13]:
test_structures = []
for pdb in os.listdir("SCOP_Test/"):
    pdb_path = os.path.join("SCOP_Test", pdb)
    parser = PDB.PDBParser()
    structure = parser.get_structure(pdb, pdb_path)
    test_structures.append(structure)

print(len(test_structures))

124


In [6]:
# OPTIONAL!! Filtering fold groups
selected_folds = list(filter(lambda x: x[1]>2, map(lambda x: (x[0],len(x[1])), fold_dict.items())))
len(selected_folds)

626

In [16]:
a=0
for fold,pdb_list in fold_dict.items():
    structures = []
    if len(pdb_list)>=2: # for stratified sampling
        for pdb in pdb_list[:5]: # at least 5 pdbs for each fold
            a+=1
print(a)

3186


In [ ]:
structures = []

for fold,pdb_list in selected_folds:
    for pdb in pdb_list[:5]:
        pdb_path = os.path.join("PDBs", pdb+".pdb")
        parser = PDB.PDBParser()
        structure = parser.get_structure(pdb, pdb_path)
        structures.append(structure)
    

In [ ]:
# Different strategies, filter sizes and encoding size

results = {}
for strategy in ["strategy1","strategy2","strategy3"]:
    results[strategy] = []
    for filter_size in [32,64,96,128,256]:
        
        structures = []
        for fold,pdb_list in selected_folds:
            for pdb in pdb_list[:5]:
                pdb_path = os.path.join("PDBs", pdb+".pdb")
                parser = PDB.PDBParser()
                structure = parser.get_structure(pdb, pdb_path)
                sstructures.append(structure)
        
        if strategy == "strategy1":
            # Resize
            train_matrix = DistanceMatrixDict(structures, resize_strategy="strategy1", resize_to=(filter_size,filter_size),removeSymmetry=True)
            test_matrix = DistanceMatrixDict(test_structures, resize_strategy="strategy1", resize_to=(filter_size,filter_size),removeSymmetry=True)
            
        elif strategy == "strategy2":
            train_matrix = DistanceMatrixDict(structures, resize_strategy="strategy2", resize_to=(filter_size,filter_size),sample_size=60)
            test_matrix = DistanceMatrixDict(test_structures, resize_strategy="strategy2", resize_to=(filter_size,filter_size),sample_size=60)

        elif strategy == "strategy3":
            train_matrix = DistanceMatrixDict(structures, resize_strategy="strategy3", resize_to=(filter_size,filter_size))
            test_matrix = DistanceMatrixDict(test_structures, resize_strategy="strategy3", resize_to=(filter_size,filter_size))

        for encoding_size in [50,100,200,500]:
            train_pdb_names, train_features = list(train_matrix.keys()), list(train_matrix.values())
            test_pdb_names, test_features = list(test_matrix.keys()), list(test_matrix.values())
            input_size = len(train_features[0])
            
            # AutoEncoder
            autoencoder = LinearAutoEncoder(input_size, encoding_size, n_iteration=100, learning_rate = 0.0001,model_path = "models/autoencoder.ckpt")
            autoencoder.train(train_features)
            
            # Embedding vectors of train and test set
            new_train_features = autoencoder.encode(train_features)
            new_test_features = autoencoder.encode(test_features)      
            
            # Prepare train X and y
            train_feature_dict = {}
            for i in enumerate(train_pdb_names):
                if "sample" in i[1]:
                    pdb = i[1].split("sample")[0]
                else:
                    pdb = i[1]
                train_feature_dict.setdefault(pdb,[])
                train_feature_dict[pdb].append(new_train_features[i[0]])
                
            # Preparing test X and y
            test_feature_dict = {}
            for i in enumerate(test_pdb_names):
                if "sample" in i[1]:
                    pdb = i[1].split("sample")[0]
                else:
                    pdb = i[1]
                test_feature_dict.setdefault(pdb,[])
                test_feature_dict[pdb].append(new_test_features[i[0]])

            X_train = []
            y_train = []
            
            for pdb,vector in train_feature_dict.items():
                X_train.append(np.average(vector,axis=0))
                y_train.append(".".join(label_dict[pdb].split(".")[:2]))
                
            X_test = []
            y_test = []
            
            for pdb,vector in test_feature_dict.items():
                X_test.append(np.average(vector,axis=0))
                y_test.append(".".join(test_label_dict[pdb.split(".")[0]].split(".")[:2]))
                
                
            uniques = list(set(y_train).union(set(y_test)))
            group2id = dict(zip(uniques, range(len(uniques))))

            X_train = np.array(X_train)
            y_train = np.array(list(map(lambda x: group2id[x], y_train)))
            X_test = np.array(X_test)
            y_test = np.array(list(map(lambda x: group2id[x], y_test)))


            sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.40, random_state=42)
            for a, b in sss1.split(X_train, y_train):
                X_train_, y_train_ = X_train[a], y_train[a]
                X_validation, y_validation = X_train[b], y_train[b]

                # Hyperparameter Optimization with validation set
                params = {'max_depth':[3,4,5,6,7,8,9,10,15,20], 
                      'criterion':('gini', 'entropy'), 
                      'warm_start':(True,False),
                     'n_estimators': (10,50,100,200,500)}

                rf = RandomForestClassifier(random_state=42)
                clf = GridSearchCV(rf, params, cv=2, refit=True)
                clf.fit(X_validation, y_validation)

                # Training best model with train set
                model = clf.best_estimator_
                model.fit(X_train_, y_train_)

                # Train and Test Accuracy Scores
                train_acc = model.score(X_train_,y_train_)
                test_acc = model.score(X_test,y_test)

                # Saving Results
                results[strategy].append(((filter_size,encoding_size,input_size), (train_acc,test_acc)))
                print(((filter_size,encoding_size,input_size), (train_acc,test_acc)))

In [ ]:
with open('pickle files/results.pkl', 'w') as f:
    pickle.dump(f, results)